In [1]:
# third-party
import pickle
import pandas as pd
import plotly.io as pio   
pio.kaleido.scope.mathjax = None

# local
from results_generator import write_results
from get_data import get_participant_ids


In [2]:
from get_data import get_data_by_id_activity

acquisition_folderpath = 'Aquisicao'
id_participants = get_participant_ids(acquisition_folderpath)
data, data_raw = get_data_by_id_activity(acquisition_folderpath)



Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [3]:
print('Writing results...')
for id in id_participants:
    print(' ---------',id,'---------------')
    write_results(id, data[id], data_raw[id], acquisition_folderpath, show_fig=False)

Writing results...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
ALR
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [4]:
thrs_comparison = pd.DataFrame({
    "thr": [0.1, 0.2, 0.3, 0.4, 0.5],
    "Ratio": [1.1, 1.1, 1.1, 1.1, 1.1],
    "Precision": [77, 84, 88, 89, 89],
    "Recall": [85, 93, 96, 97, 98],
    "tI": [13.9, 16.24, 18.18, 19.1, 19.4],
    "tI error SD": [15.84, 16.18, 17.55, 18.25, 18.79],
    "tE": [13.82, 15.51, 17.22, 17.86, 18.20],
    "tE error SD": [15.31, 15.39, 16.08, 16.97, 17.70],
    "tB": [5.45, 6.51, 7.40, 8.08, 8.26],
    "tB error SD": [7.07, 7.79, 8.82, 9.75, 10.13]
})
thrs_comparison


,thr,Ratio,Precision,Recall,tI,tI error SD,tE,tE error SD,tB,tB error SD
0,0.1,1.1,77,85,13.90,15.84,13.82,15.31,5.45,7.07
1,0.2,1.1,84,93,16.24,16.18,15.51,15.39,6.51,7.79
2,0.3,1.1,88,96,18.18,17.55,17.22,16.08,7.40,8.82
3,0.4,1.1,89,97,19.10,18.25,17.86,16.97,8.08,9.75
4,0.5,1.1,89,98,19.40,18.79,18.20,17.70,8.26,10.13


In [5]:
import numpy as np

print("Coefficient of variation\n")
for metric in ["Ratio", "Precision", "Recall", "tI", "tE", "tB", "tI error SD", "tE error SD", "tB error SD"]:
    print(f"{metric}: {(np.std(thrs_comparison[metric]) / np.mean(thrs_comparison[metric].mean()))*100:.1f}")

Coefficient of variation

Ratio: 0.0
Precision: 5.4
Recall: 5.0
tI: 11.8
tE: 9.9
tB: 14.6
tI error SD: 6.6
tE error SD: 5.7
tB error SD: 13.2


In [6]:
import plotly.graph_objects as go
from constants import CATEGORICAL_PALETTE
from plotly.subplots import make_subplots

fig = make_subplots(cols=2, 
                    x_title="Ratio of tB(last 5 breaths)",
                    y_title="Performance / Relative error (%)",
                    #subplot_titles=("FR detection", "Respiratory parameters")
    )


for i,metric in enumerate(["Precision", "Recall"]):
    
    fig.add_trace(
        go.Scatter(
            x=thrs_comparison['thr'],
            y=thrs_comparison[metric],
            line=dict(color=CATEGORICAL_PALETTE[i], width=2),
            mode='lines',
            name=metric,
        ), 
        row=1, col=1)


for i,param in enumerate(["tI", "tE", "tB"]):

    fig.add_trace(
        go.Scatter(
            x=thrs_comparison['thr'],
            y=thrs_comparison[param],
            line=dict(color=CATEGORICAL_PALETTE[i+3], width=2),
            mode='lines',
            name=param,
            # error_y=dict(
            #     type='data', # value of error bar given in data coordinates
            #     array=thrs_comparison[f"{param} SD"],
            #     visible=True
            # )
    ), 
    row=1, col=2)

fig.update(layout_showlegend=True, layout_height=400, layout_width=600)

fig.update_layout(title="The effect of different acceptability thresholds")


fig.show()
fig.write_image(f"Results/thr_performance.pdf")

In [7]:
with open('Results/results.pickle', 'rb') as file:
    data = pickle.load(file)

activities_list = ['SNB', 'SGB', 'MCH', 'SQT', 'MIXB', 'STNB', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR']

In [8]:
overview = pd.DataFrame(columns=["participant_id", "activity", "FR events", "complete cycles", "BR"])


for id in id_participants:
    print('---------',id,'---------------')
    
    new_entry = {"participant_id": id}

    for activity in activities_list:
        try:
            new_entry["activity"] = activity
            new_entry["FR events"] = len(data[id][activity]["Airflow"]["peaks"]) + len(data[id][activity]["Airflow"]["valleys"])
            new_entry["complete cycles"] = len(data[id][activity]["Airflow"]["tB (s)"])
            new_entry["BR"] = data[id][activity]["Airflow"]["BR (bpm)"]

            overview.loc[len(overview)] = new_entry
        
        except KeyError as e:
            print(f"    Could not find key {e}")
            continue

overview

--------- 7OYX ---------------
--------- NO15 ---------------
--------- G8B7 ---------------
--------- EPE2 ---------------
--------- HAK8 ---------------
--------- 1BST ---------------
--------- 83J1 ---------------
--------- QMQ7 ---------------
    Could not find key 'MIXB'
--------- 9TUL ---------------
--------- FTD7 ---------------
--------- Y6O3 ---------------
--------- 2QWT ---------------
--------- F9AF ---------------
--------- P4W9 ---------------
--------- W8Z9 ---------------
--------- D4GQ ---------------


,participant_id,activity,FR events,complete cycles,BR
0,7OYX,SNB,29,13,13.026052
1,7OYX,SGB,13,5,5.818464
2,7OYX,MCH,22,10,9.789525
3,7OYX,SQT,24,11,11.909058
4,7OYX,MIXB,24,11,16.228178
...,...,...,...,...,...
234,D4GQ,UAL,22,10,10.750762
235,D4GQ,UAR,21,9,10.217597
236,D4GQ,SE,21,10,10.209290
237,D4GQ,SS,19,9,9.303928


In [9]:
# get data where activity is not UALm, UARm, SNBm
removed_middle = overview[~overview["activity"].isin(["UALm", "UARm", "SNBm"])]
print(f"mean BR: {removed_middle['BR'].mean()}")
print(f"FR events: {removed_middle['FR events'].sum()}")
print(f"complete cycles: {removed_middle['complete cycles'].sum()}")

mean BR: 14.65143676191543
FR events: 7209
complete cycles: 3363


In [10]:
overview_by_activity = pd.DataFrame(columns=["activity", "BR [bpm]", "FR events", "complete cycles"])

for activity in activities_list:
    new_entry = {"activity": activity}
    activity_data = overview[overview["activity"] == activity]
    new_entry["FR events"] = activity_data["FR events"].sum()
    new_entry["complete cycles"] = activity_data["complete cycles"].sum()
    new_entry["BR [bpm]"] = f'{np.round(np.mean(activity_data["BR"]), 1)} $\pm$ {np.round(np.std(activity_data["BR"]), 1)}'
    overview_by_activity.loc[len(overview_by_activity)] = new_entry

overview_by_activity

,activity,BR [bpm],FR events,complete cycles
0,SNB,13.4 $\pm$ 3.4,451,208
1,SGB,5.9 $\pm$ 0.1,195,80
2,MCH,15.2 $\pm$ 4.7,504,236
3,SQT,16.5 $\pm$ 4.6,541,254
4,MIXB,12.2 $\pm$ 3.0,354,161
5,STNB,13.2 $\pm$ 3.4,455,214
6,AAL,16.5 $\pm$ 3.8,542,255
7,AAR,16.5 $\pm$ 3.9,538,252
8,ALL,17.4 $\pm$ 4.2,569,267
9,ALR,17.1 $\pm$ 4.1,565,266
